In [10]:
from keras.applications import vgg16
import cv2
from keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model

In [2]:
model = vgg16.VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3) )

In [3]:
model.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>

In [4]:
model.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [5]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model.layers[1].trainable

True

In [7]:
for i in model.layers:
    i.trainable = False

In [11]:
layer = Flatten()(model.output)
layer = Dense(units=1024, activation='relu')(layer)
layer = Dropout(0.3)(layer)
layer = Dense(units = 512, activation='relu')(layer)
layer = Dropout(0.3)(layer)
layer = Dense(units=512, activation='relu')(layer)
layer = Dropout(0.3)(layer)
layer = Dense(units=4, activation='softmax')(layer)

In [12]:
new_model = Model(inputs = model.input, outputs = layer)

In [13]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'data/train/cropped/',
        target_size=(224, 224),
        batch_size=12,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'data/test/cropped/',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

Found 779 images belonging to 4 classes.
Found 212 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

In [17]:
new_model.fit(train_generator,
                        steps_per_epoch=779/12,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=212/8
                        )

Epoch 1/10


64/64 [==============================] - 75s 1s/step - loss: 2.1686 - accuracy: 0.5379 - val_loss: 0.4496 - val_accuracy: 0.8632
Epoch 2/10
64/64 [==============================] - 70s 1s/step - loss: 0.4535 - accuracy: 0.8267 - val_loss: 0.3117 - val_accuracy: 0.9151
Epoch 3/10
64/64 [==============================] - 70s 1s/step - loss: 0.2415 - accuracy: 0.9076 - val_loss: 0.1672 - val_accuracy: 0.9528
Epoch 4/10
64/64 [==============================] - 73s 1s/step - loss: 0.2722 - accuracy: 0.8986 - val_loss: 0.2687 - val_accuracy: 0.8915
Epoch 5/10
64/64 [==============================] - 74s 1s/step - loss: 0.1851 - accuracy: 0.9294 - val_loss: 0.3794 - val_accuracy: 0.8726
Epoch 6/10
64/64 [==============================] - 73s 1s/step - loss: 0.1527 - accuracy: 0.9499 - val_loss: 0.3762 - val_accuracy: 0.8679
Epoch 7/10
64/64 [==============================] - 73s 1s/step - loss: 0.1905 - accuracy: 0.9422 - val_loss: 0.2353 - val_accuracy: 0.9104
Epoch 8/10
64/64 [

In [18]:
new_model.get_weights

<bound method Model.get_weights of <keras.src.engine.functional.Functional object at 0x000001A667B2FB50>>

In [ ]:
import json
with open("class_dictionary.json","w") as f:
    f.write(json.dumps(train_generator.class_indices))

In [155]:
import numpy as np 
from keras.preprocessing import image
img = image.load_img('./data/test/cropped/ms_dhoni/ms_dhoni11.png')
img = img.resize((224,224))
array_img = image.img_to_array(img)
fimg = np.expand_dims(array_img, axis = 0)
fimg = preprocess_input(fimg)

In [156]:
train_generator.class_indices

{'alexandra_daddario': 0, 'johnny depp': 1, 'ms_dhoni': 2, 'narendra_modi': 3}

In [158]:
y = new_model.predict(fimg)

1/1 [==============================] - 0s 86ms/step


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(resized_dhoni,cmap='gray')

In [ ]:
img = image.load_img('modi.jpg')
img = img.resize((224,224))
array_img = image.img_to_array(img)
# import numpy as np
fimg = np.expand_dims(array_img, axis = 0)

In [ ]:
faces_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye.xml')

def get_croped_image(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face = faces_cascade.detectMultiScale(img)
        for (x,y,w,h) in face:
            croped_gray = img[y:y+h, x:x+w]
            eyes = eyes_cascade.detectMultiScale(croped_gray)
            if len(eyes) >=2:
                return croped_gray

In [ ]:
dhoni_img = get_croped_image('/Projects/Celebrity_classifier/johnny.jpg')

In [ ]:
resized_dhoni = cv2.resize(dhoni_img, (224,224))

In [ ]:
resized_dhoni.shape

In [ ]:
array_img = image.img_to_array(resized_dhoni)
array_img.shape

In [ ]:
fimg = np.expand_dims(array_img, axis = 0)

In [ ]:
fimg.shape

In [ ]:
new_model.predict(fimg)

In [ ]:
train_generator.class_indices

In [28]:
from keras.models import load_model

In [29]:
x = load_model('./artifacts/model.h5')

In [150]:
x.predict(fimg)*100

1/1 [==============================] - 0s 87ms/step


array([[  0.,   0.,   0., 100.]], dtype=float32)